In [13]:
# Tratamento de Dados Programação de MP

import pandas as pd
from datetime import datetime, timedelta
import openpyxl
import warnings
import numpy as np
import pytz
warnings.filterwarnings("ignore")


# Teste de Dashboard em App

import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import ipywidgets as widgets


import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly
import plotly.graph_objects as go

import time
import streamlit as st
import plotly.express as px

# Importando as base de dados

prog_df = pd.read_excel(r'G:\Industrial\Planejamento e Controle da Produção\Matérias-Primas\Controle de M.P 2021.xlsx', sheet_name = 'Controle de M.P', engine= 'openpyxl')


In [14]:
# Tratamento de Dados 
# Selecionando as colunas

prog_df = prog_df[['Data Programado','Descrição','Seção', 'Horário Programado', 'Data Chegada', 'Hora Chegada','obs:']]

# Eliminando os valores nulos
prog_df = prog_df.dropna(how='all')

#Apagando os Dados Em branco das Linhas

prog_df.drop(prog_df[prog_df['Data Programado'].isnull()].index, inplace=True)

# Renomeando as Colunas

prog_df = prog_df.rename(columns = {'Data Programado': 'DATA PROGRAMADA', 'Descrição': 'MATÉRIA PRIMA','Seção': 'SETOR','Horário Programado':'HORÁRIO PROGRAMADO','Data Chegada':'DATA APRESENTAÇÃO', 'Hora Chegada':'HORA APRESENTAÇÃO','obs:':'EVENTUALIDADE'})

# Mudando os tipos de variáveis datetime

prog_df['DATA PROGRAMADA'] = pd.to_datetime(prog_df['DATA PROGRAMADA'], format= '%Y-%m-%d', errors= 'coerce')
prog_df['DATA APRESENTAÇÃO'] = pd.to_datetime(prog_df['DATA APRESENTAÇÃO'], format= '%Y-%m-%d', errors= 'coerce')

# Criando as variáveis de tempo compostas

prog_df['apresentação'] = prog_df['DATA APRESENTAÇÃO'].astype(str) + ' ' + prog_df['HORA APRESENTAÇÃO'].astype(str)
prog_df['apresentação'] = pd.to_datetime(prog_df['apresentação'], format= '%Y-%m-%d %H:%M:%S', errors= 'coerce')
prog_df['programação'] = prog_df['DATA PROGRAMADA'].astype(str) + ' ' + prog_df['HORÁRIO PROGRAMADO'].astype(str)
prog_df['programação'] = pd.to_datetime(prog_df['programação'], format= '%Y-%m-%d %H:%M:%S', errors= 'coerce')

#prog_df.info()

# Criando a Coluna para Tipo de Veiculo

prog_df['tipo'] = 'Silo'

# definindo as funções temporais e corrigindo o dia começando as 6AM

mytz = pytz.timezone('US/Alaska')
hoje = pytz.utc.localize(datetime.utcnow(), is_dst = None).astimezone(mytz)
hoje = str(hoje)
hoje = hoje[:-6]
hoje = pd.to_datetime(hoje, format= '%Y-%m-%d %H:%M:%S')
fim_dia = str(hoje.normalize())[:-9]
fim_dia = pd.to_datetime(fim_dia + ' ' + '23:59:59', format= '%Y-%m-%d %H:%M:%S')

agora = str(datetime.today()).split()[1]
agora = agora[:-7]
agora = datetime.strptime(agora, '%H:%M:%S')

base_df = prog_df[(prog_df['DATA APRESENTAÇÃO'] >= (hoje - timedelta(days = 2)))]

#print(agora)
#print(hoje)
#print(fim_dia)

In [15]:
## Definindo o dataframe para -tempo de atraso-

apresent_prog_df = base_df[base_df['DATA APRESENTAÇÃO'].notnull()]
apresent_prog_df['HORA APRESENTAÇÃO'] = pd.to_datetime(apresent_prog_df['HORA APRESENTAÇÃO'], format= '%H:%M:%S', errors= 'coerce')
apresent_prog_df['tempo_atraso_prog'] = apresent_prog_df['programação'] - apresent_prog_df['apresentação']
apresent_prog_df = apresent_prog_df.drop(['DATA APRESENTAÇÃO', 'HORA APRESENTAÇÃO', 'DATA APRESENTAÇÃO', 'DATA APRESENTAÇÃO', 'HORA APRESENTAÇÃO'], axis= 1)

apresent_prog_df = apresent_prog_df.sort_values(by = ['tempo_atraso_prog'], axis=0, ascending = False, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)


# Criando o ranking de Atraso

apresent_prog_df['ranking'] = apresent_prog_df['tempo_atraso_prog'].rank(axis= 0, method= 'first', ascending=True, pct=False).astype(int)
apresent_prog_df['ranking2'] = apresent_prog_df['tempo_atraso_prog'].nlargest(5, keep = 'all')
#display(apresent_prog_df)

# Agrupando por contagem o número de veículos atrasado apresentação -por seção e geral-

count_prog_df = apresent_prog_df.groupby(['SETOR', 'MATÉRIA PRIMA','ranking2']).size().reset_index(name='Atraso')
count_prog_df = pd.pivot_table(count_prog_df, index=['SETOR', 'MATÉRIA PRIMA', 'ranking2'],values='Atraso', fill_value = 0, dropna = False, aggfunc = np.sum)


display(count_prog_df)
#display(len(apresent_prog_df))
#display(prog_df)

Atraso
SETOR MATÉRIA PRIMA   ranking2               
5     AREIA 00        0 days 08:46:00       0
                      0 days 12:52:00       0
                      0 days 14:56:00       0
                      1 days 08:44:00       0
      AREIA AQ 50-800 0 days 08:46:00       1
                      0 days 12:52:00       1
                      0 days 14:56:00       0
                      1 days 08:44:00       0
7     AREIA 00        0 days 08:46:00       0
                      0 days 12:52:00       1
                      0 days 14:56:00       1
                      1 days 08:44:00       1
      AREIA AQ 50-800 0 days 08:46:00       0
                      0 days 12:52:00       0
                      0 days 14:56:00       0
                      1 days 08:44:00       0

In [16]:
# Agrupando a Quantidade de Veiculos Programados no Dia (Teste)

base2_df = prog_df[(prog_df['DATA PROGRAMADA'] >= hoje)]
count_prog_dia_df = base_df[base_df['DATA PROGRAMADA'].notnull()]

count_prog_dia_df = count_prog_dia_df.groupby(['MATÉRIA PRIMA', 'SETOR']).size().reset_index(name='Quant. Total')
display(count_prog_dia_df)

,MATÉRIA PRIMA,SETOR,Quant. Total
0,AREIA 0,5,4
1,AREIA 0,7,1
2,AREIA 00,7,6
3,AREIA 4/400,7,4
4,AREIA AQ 120,4,1
5,AREIA AQ 120,2A,2
6,AREIA AQ 50-800,5,6
7,CIMENTO BRANCO CIMSA,4,2
8,CIMENTO BRANCO CIMSA,5,2
9,CIMENTO BRANCO CIMSA,7,1


In [ ]:
# Teste de dashboard

@st.cache
def load_dataset(data_link):
    dataset = pd.read_csv(data_link)
    return dataset

st.title('Dashboard de Descarga')

@st.cache
def load_dataset(data_link):
    dataset = pd.read_csv(data_link)
    return dataset

dash_online=pd.read_csv('http://127.0.0.1:8050/')
dfash_online.head()

dfO= df_online[['name','matched_stories']]

dfO= dfO.groupby("name")["matched_stories"].sum()
dfO = pd.DataFrame(dfO)

dfToShow= dfO.sort_values("matched_stories", ascending=False)
dfToShow.head()

#titanic_link = r'C:\Users\M8949279\Desktop\Backup Projetos\Projeto descarga de matéria prima'
#titanic_data = load_dataset(titanic_link)

#st.dataframe(titanic_data)


#wid_secao = widgets.Dropdown(name= 'DATA PROGRAMADA')
#wid_secao.options = list(base_df['DATA PROGRAMADA'])
#wid_secao


In [77]:
visual_tmp = prog_df[prog_df['DATA PROGRAMADA'] == wid_secao.value].copy()
#visual_tmp.head()

In [ ]:
#Incluindo uma Figura como cartão

fig, ax = plt.subplots(figsize=(5, 1))
text = fig.text(0.5, 0.5, 'SETOR',ha='center', va='center', size=20)
plt.axis('off')
text.set_path_effects([path_effects.Normal()])

app = dash.Dash() 
# Criando a aplicação

# Criando o layout
app.layout = html.Div([
    # adicionando um titulo
        html.H1('display(count_prog_dia_df)', 
            style = {'textAlign': 'center', # alinhando o titulo ao centro
                        'fontFamily': 'Roboto', # alterando a fonte do H1
                         'paddingTop': 20}), # adicionando um padding no top
        ])

plt.show()

if __name__ == '__main__':
    app.run_server(debug = True, use_reloader = False)


In [ ]:
fig = go.Figure()
data= [go.Bar(x=[1, 2, 3], y=[1, 3, 2])]
layout= go.Layout()
title= go.layout.Title(text="A Figura")

fig.show()

In [7]:
# Teste de dashboard

style = {'description_width': 'initial'}

unique_district = df.PdDistrict.unique()

category = widgets.SelectMultiple()
    options = unique_cat.tolist()
    value = ['SETOR', 'MATÉRIA PRIMA', 'HORÁRIO PROGRAMADO']
    description='Agendamento',
    disabled=False,
    style=style,
    layout = Layout(width='50%', height='80px')
    


IndentationError: unexpected indent (<ipython-input-7-095e7eb57986>, line 8)